# Tutorial Part 1

**Goal** Open a root file and inspect its contents (trees, variables, etc) Quickly plot some variables. Histograms

First import ROOT stuff in your session 

In [ ]:
import ROOT
from ROOT import TCanvas, TF1


some black magic to show canvas you've produced

In [ ]:
%jsroot on

Open the root file

In [ ]:
f = ROOT.TFile.Open("~/data/root_session/mc_147771.Zmumu.root")

Now you can inspect it. What is the content of this file? Usually you can visualize it with a TBrowser and you will see something like that. <br>

![TBrowser TTree](images/tbrowser_mini_tree.png)
But here it doesn't work. Instead we will use
    `f.ls()`


In [ ]:
f.ls()


What's in the file?
Now access the TTree

In [ ]:
t1=f.mini

In [ ]:
t1.ls()

Now look inside the variables

In [ ]:
t1.Print()

What's the difference between vxp_z and eventNumber? What's the difference between jet_n and jet_E ?

Try to understand what are these variables.

In [ ]:
t1.Scan("eventNumber:vxp_z:lep_n:lep_pt","","colsize=5 precision=3",10)

Or you can plot some of these variables

In [ ]:
c1 = ROOT.TCanvas("c1","My Canvas",400,400)
t1.Draw("lep_pt")

In [ ]:
c1.Draw()

What units are used for the momentum? Is it a sensible histogram? What can I do to improve it? You can interact with your plot right clicking.

In [ ]:
t1.Draw("lep_pt","lep_pt<500000")
c1.Draw()

What's the difference with:

t1.Draw("lep_pt[1]","lep_pt[1]<500000")
c1.Draw()

Let's produce a one dimensional histogram

In [ ]:
from ROOT import TH1F
h1 = TH1F("h1","Leading Lepton Pt",100,0,100000)
t1.Draw("lep_pt[0]>>h1")
h1.Draw()
c1.Draw()

You can also do some "simple arithmetics" while plotting. For example, you want to change units in GeV from MeV.

In [ ]:
h2 = TH1F("h2","Leading Lepton Pt in GeV",100,0,100)
t1.Draw("lep_pt[0]/10000>>h2")
h2.Draw()
c1.Draw()

## Exercise

Produce an histogram showing the transverse energy of all the jets (GeV units)

## Saving and retrieving objects

Histograms acne be printed, saved in various formats but it would be great if they can be saved in a file and retrieved later. Histograms and objects can be saved also with directories. 

In [ ]:
outputFile = ROOT.TFile("plots/output.root", "RECREATE")
h2.Write()
outputFile.Close()

a new file appeared in the directory. 
Now retrieve the histogram from the file. With  `TBrowser` you would have seen something like that:
![TBrowser TH1](images/histo_datamuon.png)
<br>
We will use
    `my_file.ls()` as before

In [ ]:
my_file = ROOT.TFile("plots/output.root","READ")
my_file.ls()

In [ ]:
h = my_file.h1
h.Draw()
c1.Draw()

## How to save a subsample of the original file

The original root file that contains the event. Fo developing purposes (or because you are not interested in all the events) you might want to produce a lightweight ntuple by just selecting a subsample  of events. <bf>
So what you have to do is to:
<ol>
        <li> open the original file</li> 
        <li> loop on the events </li>
        <li> select some of them </li>
        <li> write only selected events to the new file</li>
</ol>

In [ ]:
nevt_out = 1000
from ROOT import TFile, TTree
my_random = ROOT.TRandom3()
my_file =TFile("~/data/root_session/DataMuons.root")
my_tree = my_file.Get("mini")
nentries = my_tree.GetEntries()
# activate all branches
my_tree.SetBranchStatus("*", 1)
my_new_file =TFile("~/data/root_session/DataMuons_10000.root", "recreate")
newtree = my_tree.CloneTree(0)
for i in range(0, nentries):
# do something smaller
#for i in range(0, nevt_out)
    my_tree.GetEntry(i)
    # doing something clever
    if my_random.Rndm()<(nevt_out/nentries):
        newtree.Fill()
        

newtree.Print()
my_new_file.Write()
my_new_file.ls()

How many events do you have in the newer file? How many events the original file was containing? Is there another possibility to reduce the size of the file?